<a href="https://www.kaggle.com/code/rattans/linear-regression-beatsperminute-simple-model?scriptVersionId=259483568" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Welcome to this notebook
> This is a simple submission
> 
> The data is not much explored
> 
> A simple linear regression model is applied over raw data

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e9/sample_submission.csv
/kaggle/input/playground-series-s5e9/train.csv
/kaggle/input/playground-series-s5e9/test.csv


# Loading the Dataset

In [2]:
X_full = pd.read_csv("../input/playground-series-s5e9/train.csv")
X_test = pd.read_csv("../input/playground-series-s5e9/test.csv")

sample = pd.read_csv("../input/playground-series-s5e9/sample_submission.csv")

In [3]:
X_full.head()

,id,RhythmScore,AudioLoudness,VocalContent,AcousticQuality,InstrumentalScore,LivePerformanceLikelihood,MoodScore,TrackDurationMs,Energy,BeatsPerMinute
0,0,0.603610,-7.636942,0.023500,0.000005,0.000001,0.051385,0.409866,290715.6450,0.826267,147.53020
1,1,0.639451,-16.267598,0.071520,0.444929,0.349414,0.170522,0.651010,164519.5174,0.145400,136.15963
2,2,0.514538,-15.953575,0.110715,0.173699,0.453814,0.029576,0.423865,174495.5667,0.624667,55.31989
3,3,0.734463,-1.357000,0.052965,0.001651,0.159717,0.086366,0.278745,225567.4651,0.487467,147.91212
4,4,0.532968,-13.056437,0.023500,0.068687,0.000001,0.331345,0.477769,213960.6789,0.947333,89.58511


In [4]:
X_full.describe()

,id,RhythmScore,AudioLoudness,VocalContent,AcousticQuality,InstrumentalScore,LivePerformanceLikelihood,MoodScore,TrackDurationMs,Energy,BeatsPerMinute
count,524164.000000,524164.000000,524164.000000,524164.000000,524164.000000,524164.000000,524164.000000,524164.000000,524164.000000,524164.000000,524164.000000
mean,262081.500000,0.632843,-8.379014,0.074443,0.262913,0.117690,0.178398,0.555843,241903.692949,0.500923,119.034899
std,151313.257586,0.156899,4.616221,0.049939,0.223120,0.131845,0.118186,0.225480,59326.601501,0.289952,26.468077
min,0.000000,0.076900,-27.509725,0.023500,0.000005,0.000001,0.024300,0.025600,63973.000000,0.000067,46.718000
25%,131040.750000,0.515850,-11.551933,0.023500,0.069413,0.000001,0.077637,0.403921,207099.876625,0.254933,101.070410
50%,262081.500000,0.634686,-8.252499,0.066425,0.242502,0.074247,0.166327,0.564817,243684.058150,0.511800,118.747660
75%,393122.250000,0.739179,-4.912298,0.107343,0.396957,0.204065,0.268946,0.716633,281851.658500,0.746000,136.686590
max,524163.000000,0.975000,-1.357000,0.256401,0.995000,0.869258,0.599924,0.978000,464723.228100,1.000000,206.037000


In [5]:
# Checking for Null
X_full.isna().sum()

id                           0
RhythmScore                  0
AudioLoudness                0
VocalContent                 0
AcousticQuality              0
InstrumentalScore            0
LivePerformanceLikelihood    0
MoodScore                    0
TrackDurationMs              0
Energy                       0
BeatsPerMinute               0
dtype: int64

In [10]:
X_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 524164 entries, 0 to 524163
Data columns (total 11 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   id                         524164 non-null  int64  
 1   RhythmScore                524164 non-null  float64
 2   AudioLoudness              524164 non-null  float64
 3   VocalContent               524164 non-null  float64
 4   AcousticQuality            524164 non-null  float64
 5   InstrumentalScore          524164 non-null  float64
 6   LivePerformanceLikelihood  524164 non-null  float64
 7   MoodScore                  524164 non-null  float64
 8   TrackDurationMs            524164 non-null  float64
 9   Energy                     524164 non-null  float64
 10  BeatsPerMinute             524164 non-null  float64
dtypes: float64(10), int64(1)
memory usage: 44.0 MB


All the data is in numeric form

# Importing Usefull Libraries

In [107]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

# Creating a simple basic submission

In [108]:
# separating features and labels
y = X_full.pop('BeatsPerMinute')
X_full.pop('id')

ids = X_test.pop('id')

In [109]:
"""scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X_full))
X_scaled.columns = X_full.columns
X_full = X_scaled.copy()

X_scaled = pd.DataFrame(scaler.transform(X_test))
X_scaled.columns = X_test.columns
X_test = X_scaled.copy()"""

'scaler = StandardScaler()\nX_scaled = pd.DataFrame(scaler.fit_transform(X_full))\nX_scaled.columns = X_full.columns\nX_full = X_scaled.copy()\n\nX_scaled = pd.DataFrame(scaler.transform(X_test))\nX_scaled.columns = X_test.columns\nX_test = X_scaled.copy()'

In [110]:
# splitting the data
X_train, X_val, y_train, y_val = train_test_split(X_full, y, test_size = 0.2, random_state = 0)

In [119]:
# Linear Regression Model
for i in X_train:
    model = LinearRegression()
    model.fit(X_train[[i]], y_train)
    pred = model.predict(X_val[[i]])
    print(i, ":", mean_squared_error(pred, y_val, squared = False))

RhythmScore : 26.43961174792847
AudioLoudness : 26.440297080211284
VocalContent : 26.440275362989272
AcousticQuality : 26.440426919735646
InstrumentalScore : 26.440232698991625
LivePerformanceLikelihood : 26.43986685897094
MoodScore : 26.439108678359204
TrackDurationMs : 26.44053023795575
Energy : 26.439763476887215


In [112]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [113]:
# making prediction and validating

pred = model.predict(X_val)
mean_squared_error(pred, y_val, squared = False)

26.438391570178112

# Fitting on whole train set

In [114]:
model.fit(X_full, y)

LinearRegression()

# Making Final Prediction

In [115]:
final = model.predict(X_test)

In [116]:
output = pd.DataFrame({'id' : ids, 
                       'BeatsPerMinute' : final})
output.head()

,id,BeatsPerMinute
0,524164,119.153193
1,524165,119.077647
2,524166,119.600333
3,524167,119.520155
4,524168,119.095133


In [117]:
output.to_csv('submission.csv', index = False)